In [2]:
from qiskit import *
import numpy as np
#the size of a must be exactly smaller than b by one. How to fix this?
#b2 can not make connection with carry 2
def logic_adder(qc, a, b, c):
    qc.barrier()
    n = a.size
    for i in range(n-1):
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i],b[i])
        if i!= 0:
            qc.ccx(c[i], b[i], c[i+1])
#     qc.barrier()
    if a.size - b.size == -1: #when size differene becomes 2, the last qubit of b can not be used currently.
        qc.ccx(a[n-1], b[n-1], b[n])
        qc.cx(a[n-1], b[n-1])
    
    if n!= 1:
        qc.ccx(c[n-1], b[n-1], b[n])
        qc.cx(c[n-1], b[n-1])
    
#     qc.barrier()
    for i in range(n-1):
        qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
        qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        qc.cx(c[(n-2)-i], b[(n-2)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
        qc.ccx(c[i], b[i], c[i+1])
#         qc.barrier()

In [3]:
##Logic adder test
for i in range(4):
    a = QuantumRegister(3,"a")
    b = QuantumRegister(4,"b")
    carry = QuantumRegister(a.size+1,"carry")
    cl = ClassicalRegister(4) #Classical output
    qc = QuantumCircuit(a,b, carry, cl)

    for index,j in enumerate(bin(i)[2:][::-1]):
        if j == "1":
            qc.x(a[index])
            qc.x(b[index+1])

    logic_adder(qc, a,b, carry)
    
    for j in range(b.size):
        qc.measure(b[j], cl[j])

    num_shots = 1 #Setting the number of times to repeat measurement
    backend = Aer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=num_shots)
    #Get results of program
    job_stats = job.result().get_counts()
    print(int(list(job_stats.keys())[0],2))

0
3
6
9
